In [6]:
# Importing Required Packages
import tensorflow as tf
import pandas as pd 
import numpy as np

from matplotlib import pyplot as plt
from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D
from keras.layers import Activation, Dense, Dropout, Flatten, InputLayer
from keras.layers import BatchNormalization
from keras.callbacks import TensorBoard
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray
from skimage.io import imsave

import os
import random

In [8]:
# Calling Required Images 
X = []
tpath = "Train/" # Where the training images are

for filename in os.listdir(tpath):
    X.append(img_to_array(load_img(tpath+filename)))
X = np.array(X, dtype=float)

# Set up train and test data
split = int(0.8*len(X))
Xtrain = X[:split]
Xtrain = 1.0/255*Xtrain

In [9]:
# Building the neural network (This is basically a rough version of encoder/decoder by Emil)
model = Sequential()
model.add(InputLayer(input_shape=(None, None, 1)))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', strides=2))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))
model.compile(optimizer='rmsprop', loss='mse')

In [ ]:
# Image transformer
datagen = ImageDataGenerator(
        shear_range=0.3,
        zoom_range=0.3,
        rotation_range=30,
        horizontal_flip=True)

# Generate training data 
batch_size = 3 #alter how as you see fit
def lab_conv(batch_size): # Conversion to Lab color scheme
    for batch in datagen.flow(Xtrain, batch_size=batch_size):
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0]
        Y_batch = lab_batch[:,:,:,1:] / 128
        yield (X_batch.reshape(X_batch.shape+(1,)), Y_batch) 
        
# Train model   
testpath = "Test/"   
tensorboard = TensorBoard(log_dir="output/first_run")
model.fit_generator(lab_conv(batch_size), callbacks=[tensorboard], epochs=10, steps_per_epoch=10) # set parameters as necesary 

In [ ]:
# Finding model loss by using remainder of training data
Xtest = rgb2lab(1.0/255*X[split:])[:,:,:,0]
Xtest = Xtest.reshape(Xtest.shape+(1,))
Ytest = rgb2lab(1.0/255*X[split:])[:,:,:,1:]
Ytest = Ytest / 128
print(model.evaluate(Xtest, Ytest, batch_size=batch_size)) # Showing model accuracy

In [16]:
# Now feeding actual black and white images 
coloring = []
for filename in os.listdir(testpath):
    coloring.append(img_to_array(load_img(testpath+filename)))
coloring = np.array(coloring, dtype=float)
coloring = rgb2lab(1.0/255*coloring)[:,:,:,0]
coloring = coloring.reshape(coloring.shape+(1,))

# Predicting/Coloring the black/white images
output = model.predict(coloring)
output = output * 128

# Output colorizations
for i in range(len(output)):
    current = np.zeros((256, 256, 3))
    current[:,:,0] = coloring[i][:,:,0]
    current[:,:,1:] = output[i]
    imsave("Initial Model Result/img_"+str(i)+".png", lab2rgb(current)) # Output images again in RGB and aving it in the folder